<a href="https://colab.research.google.com/github/zw2788/LocalMinimaConstruction/blob/main/GenerateCloseEnoughPoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
from typing import Tuple

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from IPython.display import Image
from torch.autograd import grad


In [45]:
# Define the neural network
class SimpleNN(nn.Module):
    def __init__(self, custom_W_0, custom_b, custom_V_0, custom_c):
        super(SimpleNN, self).__init__()

        # Ensure that the custom weights are tensors
        custom_W_0 = torch.tensor(custom_W_0, dtype=torch.float64)
        custom_b = torch.tensor(custom_b, dtype=torch.float64)
        custom_V_0 = torch.tensor(custom_V_0, dtype=torch.float64)
        custom_c = torch.tensor(custom_c, dtype=torch.float64)

        # Set the custom weights and biases
        self.W_0 = nn.Parameter(custom_W_0)
        self.b = nn.Parameter(custom_b)
        self.V_0 = nn.Parameter(custom_V_0)
        self.c = nn.Parameter(custom_c)

    def forward(self, x):
        x = F.sigmoid(torch.add(torch.matmul(x, self.W_0), self.b))
        x = F.sigmoid(torch.add(torch.matmul(x, self.V_0), self.c))
        return x

class FFNN(nn.Module):
    def __init__(self):
        super(FFNN, self).__init__()
        self.hidden = nn.Linear(2, 2)
        self.output = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.sigmoid(self.hidden(x))
        x = self.sigmoid(self.output(x))
        return x


# Custom loss function
def custom_loss(output, target):
    return -torch.mean(target * torch.log(output) + (1 - target) * torch.log(1 - output))


def compute_hessian_and_eigenvalues(model, data, target):
    """
    Compute the Hessian matrix and its eigenvalues for the weights of a neural network model.

    :param model: The neural network model.
    :param data: Input data (X).
    :param target: Target data (Y).
    :return: Hessian matrix and its eigenvalues.
    """
    # Forward pass
    output = model(data)
    # Compute loss
    loss = -torch.mean(target * torch.log(output) + (1 - target) * torch.log(1 - output))

    # First-order gradients (w.r.t weights)
    first_order_grads = torch.autograd.grad(loss, model.parameters(), create_graph=True)

    # Flatten the first-order gradients
    grads_flatten = torch.cat([g.contiguous().view(-1) for g in first_order_grads])

    # Hessian computation
    hessian = []
    for grad in grads_flatten:
        # Compute second-order gradients (w.r.t each element in the first-order gradients)
        second_order_grads = torch.autograd.grad(grad, model.parameters(), retain_graph=True)

        # Flatten and collect the second-order gradients
        hessian_row = torch.cat([g.contiguous().view(-1) for g in second_order_grads])
        hessian.append(hessian_row)

    # Stack to form the Hessian matrix
    hessian_matrix = torch.stack(hessian)

    # Compute eigenvalues
    eigenvalues, _ = torch.linalg.eig(hessian_matrix)

    return hessian_matrix, eigenvalues



data = pd.read_csv(
    "https://raw.githubusercontent.com/zw2788/LocalMinimaConstruction/main/output_ap.csv")

data.head()

# data , drop NaN values
X_raw,  Y, W_0, b, V_0, c = data[['x_2dvec']].dropna().values, data['y'].dropna().values, data[['W_0']].dropna().values, data[['b']].dropna().values, data[['V_0']].dropna().values, data[['c']].dropna().values

#convert string to array

X_raw = np.array([eval(s[0]) for s in X_raw])

W_0 = np.array([eval(s[0]) for s in W_0])

b = np.array([eval(s[0]) for s in b])

V_0 = np.array([eval(s[0]) for s in V_0])

c = np.array([eval(s[0]) for s in c])

# Standardize the input
# Leave blank to match the example in paper

# formatting
Y = Y.reshape((-1, 1))
print(X_raw)
print(Y)
print(W_0)
#print(X_raw.shape[0])
X_raw = torch.tensor(X_raw, requires_grad=True)
Y = torch.tensor(Y)
print(W_0, b, V_0, c)

nn_model = SimpleNN(W_0, b, V_0, c)
hessian_matrix_initial, eigenvalues_initial = compute_hessian_and_eigenvalues(nn_model, X_raw, Y)

print(eigenvalues_initial)
#check_local_minimum(eigenvalues_initial)

In [46]:
# Instantiate the network
model = FFNN()
# Optimizer (using simple Gradient Descent)
for param in model.parameters():
    param.data = param.data.double()
optimizer = optim.SGD(model.parameters(), lr=0.05)



# Dummy input and output data
input_data = torch.randn(10, 2)*5
print(input_data)
output_data = torch.randn(10, 1)
output_data = (output_data > 0.5).float()
# Convert input and output data to float64
input_data = input_data.double()
output_data = output_data.double()

# Training loop
max_num_epoch = 100000
prev_loss = None
for epoch in range(max_num_epoch):
    optimizer.zero_grad()   # clear the gradients
    outputs = model(input_data)  # forward pass
    loss = custom_loss(outputs, output_data)  # calculate the loss
    loss.backward()  # backward pass (compute gradients)
    optimizer.step()  # update weights

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/10000], Loss: {loss.item():.6f}')
        # Check for early stopping condition
    if prev_loss is not None:
        loss_diff = abs(prev_loss - loss.item())
        if loss_diff < 1e-7:
            print(f'Early stopping at epoch {epoch+1}, Loss: {loss.item():.6f}')
            break

    prev_loss = loss.item()

# Track weights and biases
weights_hidden = model.hidden.weight.data.numpy()
bias_hidden = model.hidden.bias.data.numpy()

weights_output = model.output.weight.data.numpy()
V_0_transposed = weights_output.T
bias_output = model.output.bias.data.numpy()

pre_hessianNN = SimpleNN(weights_hidden, bias_hidden, V_0_transposed, bias_output)


# Compute the eigenvalues of the Hessian
hessian_matrix_initial, eigenvalues_initial = compute_hessian_and_eigenvalues(pre_hessianNN, input_data, output_data)
print(eigenvalues_initial)

print("Eigenvalues of the Hessian matrix:", eigenvalues_initial)


# Prepare data for DataFrame
input_data_list = [str(row.tolist()) for row in input_data]
output_data_list = [str(row.item()) for row in output_data]
weights_hidden_list = [str(row.tolist()) for row in weights_hidden]
bias_hidden_list = [str(bias_hidden.tolist())]
V_0_transposed_list = [str(row.tolist()) for row in V_0_transposed]
bias_output_list = [str(bias_output.tolist())]

# Create DataFrame
df = pd.DataFrame({
    'x_2dvec': input_data_list + [None] * (max(len(weights_hidden_list), len(V_0_transposed_list)) - len(input_data_list)),
    'y': output_data_list + [None] * (max(len(weights_hidden_list), len(V_0_transposed_list)) - len(output_data_list)),
    'W_0': weights_hidden_list + [None] * (max(len(input_data_list), len(output_data_list)) - len(weights_hidden_list)),
    'b': bias_hidden_list + [None] * (max(len(input_data_list), len(output_data_list)) - len(bias_hidden_list)),
    'V_0': V_0_transposed_list + [None] * (max(len(input_data_list), len(output_data_list)) - len(V_0_transposed_list)),
    'c': bias_output_list + [None] * (max(len(input_data_list), len(output_data_list)) - len(bias_output_list))
})

# Save to CSV file
df.to_csv('output.csv', index=False)

print("Weights of hidden layer:", weights_hidden)
print("Bias of hidden layer:", bias_hidden)
print("Weights of output layer:", weights_output)
print("Bias of output layer:", bias_output)



tensor([[11.0898, -7.6508],
        [ 0.0565, -6.5113],
        [ 4.9975, -0.8779],
        [ 6.1208, -5.1308],
        [-5.5441, -6.2099],
        [ 1.1070,  0.1754],
        [-7.4783,  0.5124],
        [-7.9947,  0.8285],
        [-3.6224,  1.8058],
        [-3.0291,  2.4993]])
Epoch [10/10000], Loss: 0.800558
Epoch [20/10000], Loss: 0.730985
Epoch [30/10000], Loss: 0.681341
Epoch [40/10000], Loss: 0.645084
Epoch [50/10000], Loss: 0.617230
Epoch [60/10000], Loss: 0.594294
Epoch [70/10000], Loss: 0.574137
Epoch [80/10000], Loss: 0.555781
Epoch [90/10000], Loss: 0.539061
Epoch [100/10000], Loss: 0.524029
Epoch [110/10000], Loss: 0.510581
Epoch [120/10000], Loss: 0.498485
Epoch [130/10000], Loss: 0.487502
Epoch [140/10000], Loss: 0.477428
Epoch [150/10000], Loss: 0.468103
Epoch [160/10000], Loss: 0.459401
Epoch [170/10000], Loss: 0.451227
Epoch [180/10000], Loss: 0.443507
Epoch [190/10000], Loss: 0.436184
Epoch [200/10000], Loss: 0.429214
Epoch [210/10000], Loss: 0.422563
Epoch [220/100

# 新段落